In [3]:

from typing import Any

Fittable = Any
def simple_code_to_digraph(code):
    import re
    from inspect import getsource
    from graphviz import Digraph

    
    def get_code_str(code) -> str:
        if not isinstance(code, str):
            return getsource(code)
        return code
    
    empty_spaces = re.compile('^\s*$')
    simple_assignment_p = re.compile(
        '(?P<output_vars>[^=]+)'
        '\s*=\s*'
        '(?P<func>\w+)'
        '\((?P<input_vars>.*)\)'
    )
    comma_sep = re.compile('\s*,\s*')


    def parsed_lines(code_str, simple_assignment_p):
        p = simple_assignment_p
        for line in code_str.split('\n'):
            if not empty_spaces.match(line):
                line = line.strip()
                m = simple_assignment_p.search(line)
                if m:
                    yield m.groupdict()

    def parsed_lines_to_dot(parsed_lines):
        for d in parsed_lines:
            yield f"{d['input_vars']} -> {d['func']} -> {d['output_vars']}"
            
            
    code_str = get_code_str(code)
    dot_lines = parsed_lines_to_dot(parsed_lines(code_str, simple_assignment_p))
    return Digraph(body=dot_lines)


# example ##########################################################################
def user_story(
    data_source: Any,
    featurizer_learner: Fittable,
    outlier_gauger_learner: Fittable,
):
    wfs = make_wfs(data_source)
    chks = chunker(wfs)
    train_chks, test_chks = splitter(chks)

    featurizer_obj = learn_featurizer(featurizer_learner, train_chks)
    train_fvs = featurizer(featurizer_obj, train_chks)

    outlier_gauger_obj = learn_outlier_gauger(outlier_gauger_learner, train_fvs)
    train_outlier_scores = outlier_gauger(outlier_gauger_obj, train_fvs)